In [3]:
import pandas as pd
import numpy as np

In [2]:
#df_main = pd.read_csv("/Users/shashanksingh/Desktop/KENYA_LAW_DATA/METADATA_ANALYSIS/FINAL_DATA/metadata_merged_v8.csv").drop(["Unnamed: 0"], axis = 1)

In [3]:
#df_main.head()

In [4]:
df_judges = pd.read_csv("/Users/shashanksingh/Desktop/KENYA_LAW_DATA/working_data_luis/search_engine_judge_names_by_case.csv")

In [5]:
df_judges = df_judges[df_judges["normalized_name"].notnull()]

In [6]:
df_judges

,case_id,original_judge_name,normalized_name,search_engine_judge_name
0,1,amraphael mbogholi-msagha,amraphael mbogholi-msagha,amraphael mbogholi-msagha
1,10,a b shah,a b shah ja,amrittal bhagwanji shah
2,100,alnashir ramazanali magan visram,alnashir ramazanali magan visram,alnashir ramazanali magan visram
3,10000,riaga samuel cornelius omolo,riaga samuel cornelius omolo,riaga samuel cornelius omolo
4,10000,philip kiptoo tunoi,philip kiptoo tunoi,philip kiptoo tunoi
...,...,...,...,...
155881,99995,oscar angote,oscar angote,oscar angote
155882,99996,lucy waithaka,lucy waithaka,lucy waithaka
155883,99997,mumbi ngugi,mumbi ngugi,mumbi ngugi
155884,99998,maureen akinyi odero,maureen akinyi odero,maureen akinyi odero


In [7]:
df_judges = df_judges.drop_duplicates(subset='case_id', keep=False).reset_index(drop=True)

In [8]:
len(df_judges)

118326

In [9]:
list_judge_names = list(set(list(df_judges["normalized_name"])))

In [10]:
list_judge_names

['william ouko',
 'benard mweresa eboso',
 'j w mwera j',
 'alnashir ramazanali magan visram',
 'norbury dugdale',
 'w k korir',
 'mary muthoni gitumbi',
 'e k usui macharia',
 'julie oseko',
 'g m njuguna',
 'elija ogoti obaga',
 'none',
 'antony charo mrima',
 "bwonwong'a justus momanyi",
 'r k ondieki',
 'edward nii adjar torgbor',
 'm n gicheru',
 'c obulutsa',
 'olga akech sewe',
 'hellen wasilwa seruya',
 'e malesi',
 't s luvuga',
 'nelly awori matheka',
 'm warsame',
 's k gacheru',
 'monica mbaru',
 'm w wachira',
 'fred kwasi apaloo',
 'thande mugure',
 'ngaah jairus',
 'mohammed khadhar ibrahim',
 'george ernest omondi tunya',
 'm w murage',
 'eugene cotran',
 'felix kombo',
 'edward trevelyan',
 'judith wanjala',
 'david maitai rimita',
 'christine wanjiku meoli',
 'stella ngali mutuku',
 'sharad rao',
 'mwangi njoroge',
 'milicent akinyi odeny',
 'c w githua',
 'washika',
 'mohamed noor kullow',
 's ongeri',
 'mitey j k j',
 'martin muya',
 'e juma',
 'mg mugo j',
 'dolphi

In [4]:
from pathlib import Path
main_dir = Path("/u/home/i/iacir21/myscratch")
xlsx_path = main_dir / "train_test_set" / "metadata_merged_v15.xlsx"
df_2 = pd.read_excel(xlsx_path)

In [5]:
if "normalized_judges" in df_2.columns:
    print("Column exists!")
else:
    print("Column not found.")

Column exists!


In [6]:
df_judges = pd.read_csv("/u/home/i/iacir21/myscratch/train_test_set/judge_train_caseids.csv")

In [7]:
# Assume df_judges is the judge-level table you showed
df_expanded = (
    df_judges.assign(train_case_ids=df_judges["train_case_ids"].astype(str).str.split(";"))
             .explode("train_case_ids")
             .reset_index(drop=True)
)

# Make sure case IDs are strings (same type as in df_2)
df_expanded["train_case_ids"] = df_expanded["train_case_ids"].str.strip()


In [8]:
cols_to_keep = ["judge(s)", "normalized_judges", "length_of_judgement", "case_id"]
df_2_sub = df_2[cols_to_keep].copy()
df_2_sub["case_id"] = df_2_sub["case_id"].astype(str).str.strip()


In [9]:
train_ids = set(df_expanded["train_case_ids"])
df_filtered = df_2_sub[df_2_sub["case_id"].isin(train_ids)].copy()


In [10]:
df_3 = pd.read_csv("/u/home/i/iacir21/myscratch/train_test_set/median_year_judges.csv")

df_final = df_filtered.merge(
    df_3, 
    on="judge(s)", 
    how="left"   # keeps all rows in df_filtered, adds matching columns from df_3
)


In [11]:
# Basic save
df_final.to_csv("/u/home/i/iacir21/myscratch/replication/judge_normalized.csv", index=False)


In [12]:
df_final

,judge(s),normalized_judges,length_of_judgement,case_id,median_year,eligible_docs,train_docs
0,james aaron makau,james aaron makau,519,132434,2015.0,732,451
1,pauline nyamweya,pauline nyamweya,972,83775,2014.0,1230,731
2,lucy nyambura gacheru,lucy nyambura gacheru,2014,98131,2015.0,990,539
3,samson odhiambo okong'o,samson odhiambo okong'o,4317,106493,2013.0,1010,620
4,margaret waringa muigai,margaret waringa muigai,1067,141502,2014.0,816,432
...,...,...,...,...,...,...,...
82976,antonina kossy bor,antonina kossy bor,1402,150750,2014.0,443,247
82977,mumbi ngugi,mumbi ngugi,2054,98728,2014.0,667,404
82978,teresia mumbua matheka,teresia mumbua matheka,657,169222,2015.0,468,252
82979,stephen kibunja,stephen kibunja,734,122263,2014.0,831,441


In [13]:
list_judge_names=['william ouko',
 'benard mweresa eboso',
 'j w mwera j',
 'alnashir ramazanali magan visram',
 'norbury dugdale',
 'w k korir',
 'mary muthoni gitumbi',
 'e k usui macharia',
 'julie oseko',
 'g m njuguna',
 'elija ogoti obaga',
 'none',
 'antony charo mrima',
 "bwonwong'a justus momanyi",
 'r k ondieki',
 'edward nii adjar torgbor',
 'm n gicheru',
 'c obulutsa',
 'olga akech sewe',
 'hellen wasilwa seruya',
 'e malesi',
 't s luvuga',
 'nelly awori matheka',
 'm warsame',
 's k gacheru',
 'monica mbaru',
 'm w wachira',
 'fred kwasi apaloo',
 'thande mugure',
 'ngaah jairus',
 'mohammed khadhar ibrahim',
 'george ernest omondi tunya',
 'm w murage',
 'eugene cotran',
 'felix kombo',
 'edward trevelyan',
 'judith wanjala',
 'david maitai rimita',
 'christine wanjiku meoli',
 'stella ngali mutuku',
 'sharad rao',
 'mwangi njoroge',
 'milicent akinyi odeny',
 'c w githua',
 'washika',
 'mohamed noor kullow',
 's ongeri',
 'mitey j k j',
 'martin muya',
 'e juma',
 'mg mugo j',
 'dolphina alego snr',
 'b m ochoi',
 'mary muhanji kasango',
 'john mutungi',
 'janet nzilani mulwa',
 'h m ng’ang’a',
 'roselyn naliaka nambuye',
 'khapoya s benson',
 'mutitu r m j',
 'm odero',
 'william musya musyoka',
 'r k koech',
 'mwangi',
 'kosgei',
 'g k kimondo',
 'd k matutu esq',
 'r odenyo',
 "kathurima m'inoti",
 'v o adet',
 'stewart mwachiru madzayo',
 'maureen akinyi odero',
 'robert kipkoech limo',
 'abdullah',
 's m mokua',
 'langat-koech c betty',
 'rose edwina atieno ougo',
 'alex george aluri etyang',
 'james rika',
 'raj bahadar bhandari',
 'james aaron makau',
 'pj ransely j',
 'abida ali-aroni',
 'andrew isaac hayanga',
 'sila munyao',
 'nyutu',
 'd n ogoti',
 'grace a mmasi',
 'jackson kasanga mulwa',
 'jonathan bowen havelock',
 'cecilia wathaiya githua',
 'mwai',
 'daniel david ntanda nsereko',
 'hayanga a i',
 'gilbert andalya omwitsa',
 'lawrence peter ouna',
 'betty rashid',
 'akua kuenyehia',
 'jamila mohammed',
 'joseph william alexander butler-sloss',
 'akilano molade akiwumi',
 'enock chirchir cherono',
 'jeanne wanjiku gacheche',
 'daniel kiio musinga',
 'j l a osiemo j',
 'g p omondi',
 'stella munai muketi',
 's k onjoro',
 'wendy k micheni',
 'jaswinder singh sehmi',
 'george martin ongondo',
 'richard mururu mwongo',
 'gurbachan singh pall',
 'linus kassan',
 'amrittal bhagwanji shah',
 'samuel william wako wambuzi',
 'l n mugambi',
 'philip kiptoo tunoi',
 'edward muthoga muriithi',
 'francis gikonyo',
 'muthoga ca',
 'joseph raymond otieno masime',
 'j v o juma',
 'philip nyamu waki',
 's r wewa',
 'a m obura',
 'j munguti',
 'cosmas maundu',
 'r k langat',
 'norbert okumu',
 'lucy njora drsc',
 'chunilal bhagwandas madan',
 'mohammed abdullahi warsame',
 'marete d k njagi',
 'b ombewa',
 'paul kihara kariuki',
 'nicholas randa owano ombija',
 'oscar angote',
 'bernard chunga',
 'apondi m',
 'roseline a oganyo',
 'james onyiego nyarangi',
 "anne apondi ong'injo",
 'said juma chitembwe',
 'robert mugo mutitu',
 'david a onyancha',
 'mandari',
 'hpg wawaru',
 'v karanja',
 'stephen nyangau riechi',
 'john walter onyango otieno',
 'lucy waruguru gitari',
 'c',
 'david christopher porter',
 'ang’awa m a',
 'r n muriuki',
 'timothy o okello',
 'd o onyango',
 'b n olao',
 'hydfbxbas',
 'mathews nderi nduma',
 'l ambasi',
 'charles yano kimutai',
 'benjamin patrick kubo',
 'c k obara',
 'erastus mwaniki githinji',
 'james otieno olola',
 'l k mutai',
 'r nyakundi',
 'osiemo j l a',
 'c a otieno',
 'a a lakha',
 'moijo matayia ole keiwua',
 'y a shikanda',
 'margaret waringa muigai',
 'aggrey otsyula muchelule',
 'abdul majid cockar',
 'r ougo',
 'charles pius chemuttut',
 'of ingutya at wajir',
 'harold grant platt',
 'abigail mshila',
 'rachel biomondo ngetich',
 'antonina kossy bor',
 'jane muyoti onyango',
 'philip john ransley',
 'joyce nuku khaminwa',
 'barabara kiprugut tanui',
 'd a orimba',
 'dermot joseph sheridan',
 'malesi kidali',
 'reuben nyambati nyakundi',
 'loice chepkemoi komingoi',
 'a i hussein',
 'c w meoli',
 'p ngare gesora',
 'kuloba r',
 'susan ndegwa',
 'hpg',
 'teresia mumbua matheka',
 'orenge k i',
 'w j gichimu',
 'jairus ngaah',
 'anthony kaniaru',
 'pritam singh brar',
 'd k njagi marate',
 'lucy nyambura gacheru',
 'd w mburu',
 'david shikomera majanja',
 'nathan shiundu lutta',
 'kaburu bauni',
 'roseline pauline vunoro wendoh',
 'abdulla mustafa',
 'sankale ole kantai',
 'joyce adhiambo aluoch',
 'j k mitey',
 's o mogute',
 'p n maina',
 'g m a ong’ondo',
 'a c a ong’injo',
 'j gandani',
 'j kiarie',
 'b k tanui',
 'l kimaru',
 'jackton boma ojwang',
 'margaret njoki mwangi',
 'daniel kennedy sultani aganyanya',
 'mwera j w j',
 'n njagi',
 'david kenani maraga',
 'byram ongaya',
 't m mwangi',
 'boaz nathan olao',
 'a b shah ja',
 'lydia awino achode',
 'ruth nekoye sitati',
 'kalpana hasmukhrai rawal',
 'justus kituku',
 'c o nyawiri',
 "hedwig imbosa ong'udi",
 'onesmus ndambuthi makau',
 's o oguk',
 'm munyao sila',
 'b p kub0',
 'anne omollo',
 'mathew guy muli',
 'mwilu',
 'j o magori',
 'o a angote',
 's o temu',
 'joseph vitalis odero juma',
 's m mungai',
 'stephen kibunja',
 'ernest fredrick aragon',
 'tutui',
 'musyoka j',
 'b ochieng',
 'w a juma',
 'g n wakahiu',
 'e a obina',
 'florence nyaguthii muchemi',
 'r walekhwa',
 'trail',
 'c k njai',
 'benard ochieng ondego',
 'j k kingori',
 'jacqueline kamau',
 'onguto joseph louis omondi',
 'james n mwaniki',
 'jemutai grace kemei',
 'm o wambani',
 'hellen amolo omondi',
 'charles gitonga mbogo',
 'l m wachira',
 'g w ngenye â€“ macharia',
 'b mosiria',
 'pk rugut',
 'etyang a g a j',
 'james otieno odek',
 'patel v v j',
 'agnes kalekye murgor',
 't a odera',
 'joseph gregory nyamu',
 'lucy waithaka',
 'james patrick trainor',
 'james wakiaga',
 'surrender kumar sachdeva',
 'daniel ogolla',
 'peter john hewett',
 'onesmus kimweli mutungi',
 'richard otieno kwach',
 'christine atieno ochieng',
 'tank j',
 'v k kiptoon',
 'isaac charles cheskaki wambilyangah',
 'j n muniu',
 'sheikh mohammed amin',
 'm m nafula',
 "c o ong'udi",
 "samson odhiambo okong'o",
 'p m muilu',
 't obutu',
 'effie owuor',
 'caroline kemei',
 'francis tuiyott',
 'william mbaya',
 'lesiit jessie w',
 'ndwiga',
 'david kipyegomen kemei',
 'juma j v o',
 'fatuma sichale',
 'd',
 's okongôçöo',
 'e muriuki nyagah',
 'abdulrasul ahmed lakha',
 'cecil henry ethelwood miller',
 "j k ng'arng'ar",
 'm onkoba',
 'luka kiprotich kimaru',
 'john mwangi gachuhi',
 'leonard njagi',
 'mary clausina oundo',
 'p n areri',
 'richard charles namasaka kuloba',
 'esther nyambura maina',
 'james frank shields',
 'zakayo richard chesoni',
 'samwel ndungu mukunya',
 'l n waithaka',
 'm k mwangi',
 'nzioki wa makau',
 'joseph mbalu mutava',
 'm sudi',
 'wilson nkunja kaberia',
 'tanui b k arap j',
 'w kagendo',
 's lamu',
 'fidulhussein esmailji abdullah',
 'nelson jorum abuodha',
 "a g a etyang'",
 'grace lidembu nzioka',
 's k mutai',
 'wilfrida adhiambo okwany',
 'c n ndegwa',
 'mayamba c a',
 'elena g nderitu',
 'mwongo',
 'b r kipyegon',
 'patrick omwenga kiage',
 'patrick j okwaro otieno',
 's n riechi',
 'alister arthur kneller',
 'radido stephen okiyo',
 'riaga samuel cornelius omolo',
 'mumbi ngugi',
 'sarah chibai ondeyo',
 'kunyuk tito',
 'weldon kipyegon korir',
 'george matatia abaleka dulu',
 'muga apondi',
 'kiarie waweru kiarie',
 '0',
 'na',
 'farah s m amin',
 'john wycliffe mwera',
 'j j masiga',
 'george benedict maina kariuki',
 'doreen mulekyo',
 'm wakahora',
 'w k chepseba',
 'lucy ngima mbugua',
 'hazel wandere',
 'j m mutungi',
 'stephen gatembu kairu',
 'derek schofield',
 'linnet ndolo',
 "emmanuel okello o'kubasu",
 'alfred henry simpson',
 'w ouko',
 "john muting'a mativo",
 'peter muchoki njoroge',
 'b j ndeda',
 'george vincent odunga',
 'n',
 's n makila',
 'abdallah j',
 'm amin j',
 'njeri thuku',
 'rewa',
 'k sambu',
 'anthony k mwicigi',
 'william shirley deverell',
 'peter nyagaka areri',
 'johnson evan gicheru',
 "'s' at nairobi)",
 'p j ransley judge',
 "ndung'u h n",
 'pauline nyamweya',
 's m shitubi',
 'eric john ewen law',
 'wanjiru karanja',
 'temba a sitati',
 'john luka osiemo',
 'mulwa j k',
 'maureen atieno onyango',
 'e obaga',
 'eric kennedy okumu ogola',
 'm kasera',
 'aaron g ringera',
 'hilary kiplagat chemitei',
 'martha a nanzushi',
 'gideon p mbito',
 'john amonde mango',
 'ekaterina trendafilova',
 'philomena mbete mwilu',
 'amraphael mbogholi-msagha',
 'lucy mwihaki njuguna',
 'hannah magondi okwengu',
 'khamoni j m j',
 'johnson kiptonui mitey',
 'ezra o awino',
 'e a nyaloti',
 'murugi geteria mugo',
 'kanyi kimondo',
 'samwel odhiambo oguk',
 'lilian nabwire mutende',
 'andayi w francis',
 'abdul rauf samnakay',
 'isaac lenaola',
 "mary atieno ang'awa",
 'roseline lagat-korir',
 'john nyabuto onyiego',
 'john henry sydney todd',
 'm i g moranga',
 'l t lewa',
 'milton stephen asike makhandia',
 'p gichohi',
 'paul kiptenai kimisoy arap birech',
 'alan robin winston hancox',
 'stephen murugu githinji',
 'e wanjala',
 'r n kimingi',
 'e k usui',
 'hatari peter george waweru',
 'felix makoyo',
 'mutungi charles kariuki',
 'tom mbaluto',
 'b mararo',
 'bernard kasavuli',
 's m s soita',
 'joseph kiplagat sergon',
 'martha karambu koome',
 'dorah o chepkwony',
 'cheruto c kipkorir',
 'jessie wanjiku lessit',
 'm mutuku',
 'a k mokoross',
 'n a owino',
 'kenneth d potter',
 'judge',
 'beatrice thuranira jaden',
 'antony ombwayo',
 "anthony ndung'u kimani",
 'joseph raphael karanja',
 'mitei j',
 'samuel elikana ondari bosire',
 'ransley p j',
 'asenath nyaboke ongeri',
 'patrick john kamau',
 'leslie gerald eyre harris',
 'j k sergon',
 'fred andago ochieng',
 's n abuya',
 'e n maina',
 'thripsisa wanjiku cherere',
 'festus azangalala',
 'nagillah chrispin beda',
 'gilbert shikwe',
 'dalmas omondi ohungo',
 'kennedy a bidali',
 'vinubhai vithalbhai patel',
 'william kipsiro tuiyot',
 'pamela mwikali tutui',
 'l n mesa',
 'r o kwach ja',
 'enock chacha mwita',
 'bauni k j',
 'jesse nyagah njagi',
 'p mayova',
 'of kenya',
 "a b mong'are",
 'a k kaniaru',
 'm l nabibya',
 'abdulhalim h athman',
 'roselyne ekirapa aburili',
 'john micheal khamoni',
 'daniel ogola ogembo',
 'alfred mabeya',
 'mathew john anyara emukule',
 'yuvinalis maronga angima',
 't w murigi',
 'kamau pj j',
 'evans w muleka',
 'joel mwaura ngugi']

In [14]:
unique_judges_df = set(df_final["normalized_judges"].unique())
unique_judges_list = set(list_judge_names)


In [15]:
# in list_judge_names but not in df_judges["judge(s)"]
only_in_list = unique_judges_list - unique_judges_df  

# in df_judges["judge(s)"] but not in list_judge_names
only_in_df = unique_judges_df - unique_judges_list  

# intersection (common judges in both)
common = unique_judges_list & unique_judges_df  


In [16]:

print("Only in list_judge_names:", only_in_list)

Only in list_judge_names: {'g w ngenye â€“ macharia', 'ekaterina trendafilova', 'na', 'akua kuenyehia', 'daniel david ntanda nsereko', 'apondi m'}


In [17]:
print("Only in df_judges['judge(s)']:", only_in_df)

Only in df_judges['judge(s)']: {nan, 'hon. a.d. wako - senior resident kadhi', 'a. d. wako senior resident kadhi', 'hon. idris n. nyaboga resident kadhi homabay kadhi court', 'b.kimemia               -          chairman-signed.  r.mwambura          –          member-signed.  p.swanya                 -          member-signed.', 'dick waweru (chairman) prof. kiarie mwaura (member) aden noor ali (member) betty chepng’etich bii (member) moses parantai (member) betty bii (member)', 'a.ishaq hussein  senior resident kadhi', 'b. kimemia   -  chairman      f. terer  - deputy chairman   r. mwambura  -  member     p. gichuki  -   member', 'dick waweru   chairman  prof. kiarie mwaura  member  betty chepng’etich bii  member', 'hon. idris n. nyaboga resident kadhi', 'mursal m sizi kadhi', 'dick waweru  chairman prof. kiarie mwaura member  moses parantai member  aden noor ali  member betty chepng’etich bii  member', 'dick waweru   chairman     prof. kiarie mwaura  member     moses parantai   member 

In [18]:
print("Common:", common)

Common: {'hayanga a i', 't m mwangi', 'samuel william wako wambuzi', 'd k matutu esq', 'mohammed abdullahi warsame', "'s' at nairobi)", 'r ougo', 'd a orimba', 'w a juma', 'a k mokoross', 'francis tuiyott', 'alister arthur kneller', 'jonathan bowen havelock', 'said juma chitembwe', 'jamila mohammed', 's k onjoro', 'mwangi njoroge', 'o a angote', 'p m muilu', 'm w murage', 'harold grant platt', 'w j gichimu', 'd', 'alfred mabeya', 'c w meoli', 'olga akech sewe', 'mathew john anyara emukule', 'abdul rauf samnakay', 'v k kiptoon', 'trail', 'gurbachan singh pall', 'william kipsiro tuiyot', 'milton stephen asike makhandia', 'e muriuki nyagah', 'martin muya', 'mitey j k j', 'judge', "kathurima m'inoti", 'l k mutai', 'mitei j', 'kenneth d potter', 'b p kub0', 'charles gitonga mbogo', 's r wewa', 'd k njagi marate', 'joyce nuku khaminwa', 'p j ransley judge', 'linus kassan', 'john mutungi', 's o oguk', 's n riechi', 'charles pius chemuttut', 'paul kihara kariuki', 'nicholas randa owano ombija'

In [19]:
root_dir = "/u/home/i/iacir21/myscratch/train_test_set/judgements_txt_train"
target_dir = "/u/home/i/iacir21/myscratch/replication/judges_corpus"

In [20]:
import os
from tqdm import tqdm

In [21]:
for judge in tqdm(list_judge_names):
    
    df_temp = df_final[df_final["normalized_judges"] == judge]
    list_ids = list(set(list(df_temp["case_id"])))
    
    for case_id in list_ids:
        root_path = os.path.join(root_dir, str(case_id) + ".txt")
        target_path = os.path.join(target_dir, judge)
        
        if not os.path.exists(target_path):
            os.makedirs(target_path)
            
        
        f = open(root_path,"r").read()
        
        file_path = os.path.join(target_path, str(case_id) + ".txt")
        
        g = open(file_path,"w+")
        g.write(f)
        
        

100% 504/504 [08:38<00:00,  1.03s/it]


In [23]:
# Count how many unique normalized_judge values exist
num_unique_judges = df_final["normalized_judges"].nunique()
print("Number of unique normalized judges:", num_unique_judges)


Number of unique normalized judges: 578


In [25]:
import os
import pandas as pd

base_dir = "/u/home/i/iacir21/myscratch/replication/judges_corpus"   # replace with your actual path

judge_data = []

# Loop over each judge folder
for judge_folder in os.listdir(base_dir):
    judge_path = os.path.join(base_dir, judge_folder)
    
    if os.path.isdir(judge_path):
        total_words = 0
        num_cases_v2 = 0
        
        # Loop over each .txt file inside the judge's folder
        for file_name in os.listdir(judge_path):
            if file_name.endswith(".txt"):
                file_path = os.path.join(judge_path, file_name)
                with open(file_path, "r", encoding="utf-8", errors="ignore") as f:
                    text = f.read()
                    total_words += len(text.split())
                    num_cases_v2 += 1
        
        judge_data.append([judge_folder, total_words, num_cases])

# Convert to DataFrame
df_tokens = pd.DataFrame(judge_data, columns=["judge_name", "num_tokens", "num_cases_v2"])

print(df_tokens.head())


        judge_name  num_tokens  num_cases_v2
0  's' at nairobi)        3003             0
1                0        1986             0
2        a a lakha         699             0
3     a b mong'are         317             0
4      a b shah ja        7750             0


In [27]:
df_tokens

,judge_name,num_tokens,num_cases_v2
0,'s' at nairobi),3003,0
1,0,1986,0
2,a a lakha,699,0
3,a b mong'are,317,0
4,a b shah ja,7750,0
...,...,...,...
493,william shirley deverell,48533,0
494,wilson nkunja kaberia,7141,0
495,y a shikanda,16114,0
496,yuvinalis maronga angima,524763,0


In [26]:
df_final

,judge(s),normalized_judges,length_of_judgement,case_id,median_year,eligible_docs,train_docs
0,james aaron makau,james aaron makau,519,132434,2015.0,732,451
1,pauline nyamweya,pauline nyamweya,972,83775,2014.0,1230,731
2,lucy nyambura gacheru,lucy nyambura gacheru,2014,98131,2015.0,990,539
3,samson odhiambo okong'o,samson odhiambo okong'o,4317,106493,2013.0,1010,620
4,margaret waringa muigai,margaret waringa muigai,1067,141502,2014.0,816,432
...,...,...,...,...,...,...,...
82976,antonina kossy bor,antonina kossy bor,1402,150750,2014.0,443,247
82977,mumbi ngugi,mumbi ngugi,2054,98728,2014.0,667,404
82978,teresia mumbua matheka,teresia mumbua matheka,657,169222,2015.0,468,252
82979,stephen kibunja,stephen kibunja,734,122263,2014.0,831,441


In [29]:
df_thresh = df_tokens[df_tokens["num_tokens"]>=50000]

In [28]:
len(df_tokens[df_tokens["num_tokens"]>=250000])

153

In [31]:
df_thresh

,judge_name,num_tokens,num_cases_v2
11,aaron g ringera,76309,0
19,abida ali-aroni,417870,0
20,abigail mshila,426766,0
21,aggrey otsyula muchelule,657514,0
23,akilano molade akiwumi,64406,0
...,...,...,...
488,wilfrida adhiambo okwany,883215,0
489,william kipsiro tuiyot,74072,0
491,william musya musyoka,1724065,0
492,william ouko,465681,0


In [33]:
df_thresh.to_csv("/u/home/i/iacir21/myscratch/replication/df_thresh_via.csv", index=False)


In [ ]:
**************************************************************************************************

In [50]:
df_new

,judge,gender,tokens,cases
0,william ouko,['M'],841265,877
1,benard mweresa eboso,['M'],442803,236
2,j w mwera j,['M'],32947,27
3,alnashir ramazanali magan visram,['M'],581694,512
4,norbury dugdale,['M'],2570,1
...,...,...,...,...
499,yuvinalis maronga angima,['M'],444956,262
500,t w murigi,['F'],2782,1
501,kamau pj j,['M'],2496,5
502,evans w muleka,['M'],6434,3


In [61]:
len(df_new[df_new["tokens"]>=250000])

160

In [62]:
len(df_new[(df_new["tokens"]>=250000) & (df_new["gender"]=="['F']")])

67

In [63]:
df_new[(df_new["tokens"]>=250000) & (df_new["gender"]=="['F']")]

,judge,gender,tokens,cases
6,mary muthoni gitumbi,['F'],838755,623
18,olga akech sewe,['F'],1039166,407
19,hellen wasilwa seruya,['F'],1899674,1138
22,nelly awori matheka,['F'],639133,461
25,monica mbaru,['F'],2231298,775
...,...,...,...,...
454,jessie wanjiku lessit,['F'],1741387,968
460,beatrice thuranira jaden,['F'],961920,1000
474,thripsisa wanjiku cherere,['F'],574626,506
476,nagillah chrispin beda,['F'],309710,293


In [67]:
df_thresh = df_new[df_new["tokens"]>=50000]

In [68]:
df_thresh = df_thresh.reset_index(drop=True)

In [69]:
df_thresh

,judge,gender,tokens,cases
0,william ouko,['M'],841265,877
1,benard mweresa eboso,['M'],442803,236
2,alnashir ramazanali magan visram,['M'],581694,512
3,mary muthoni gitumbi,['F'],838755,623
4,elija ogoti obaga,['M'],1020921,934
...,...,...,...,...
210,john micheal khamoni,['M'],748028,466
211,alfred mabeya,['M'],1883797,1042
212,mathew john anyara emukule,['M'],1877197,909
213,yuvinalis maronga angima,['M'],444956,262


In [70]:
df_thresh.to_csv("/Users/shashanksingh/Desktop/GloVe/judges_corpus/df_thresh.csv")

In [78]:
df_temp = df_thresh.sort_values(["judge"])

In [79]:
df_temp = df_temp.reset_index(drop=True)

In [80]:
df_temp[:50]

,judge,gender,tokens,cases
0,a i hussein,['M'],50990,35
1,aaron g ringera,['M'],151630,62
2,abdulhalim h athman,['M'],62141,52
3,abdulrasul ahmed lakha,['M'],51772,78
4,abida ali-aroni,['F'],572904,483
5,abigail mshila,['F'],685048,451
6,aggrey otsyula muchelule,['M'],941053,1063
7,agnes kalekye murgor,['F'],53221,35
8,akilano molade akiwumi,['M'],87126,49
9,alex george aluri etyang,['M'],171047,96


In [81]:
df_temp[50:100]

,judge,gender,tokens,cases
50,emmanuel okello o'kubasu,['M'],235527,209
51,enock chacha mwita,['M'],981431,279
52,enock chirchir cherono,['M'],220077,152
53,erastus mwaniki githinji,['M'],434016,304
54,eric kennedy okumu ogola,['M'],2134064,942
55,esther nyambura maina,['F'],655962,443
56,farah s m amin,['F'],440377,171
57,festus azangalala,['M'],1265838,944
58,florence nyaguthii muchemi,['F'],1270137,952
59,francis gikonyo,['M'],2554526,1017


In [82]:
df_temp[100:150]

,judge,gender,tokens,cases
100,john mutungi,['M'],1567136,632
101,john nyabuto onyiego,['M'],360138,191
102,john walter onyango otieno,['M'],402377,214
103,john wycliffe mwera,['M'],782664,565
104,jonathan bowen havelock,['M'],1323364,391
105,joseph gregory nyamu,['M'],562872,205
106,joseph kiplagat sergon,['M'],2904611,2717
107,joseph mbalu mutava,['M'],203095,110
108,joseph raphael karanja,['M'],1174166,749
109,joyce adhiambo aluoch,['F'],305259,225


In [71]:
df_thresh[:50]

,judge,gender,tokens,cases
0,william ouko,['M'],841265,877
1,benard mweresa eboso,['M'],442803,236
2,alnashir ramazanali magan visram,['M'],581694,512
3,mary muthoni gitumbi,['F'],838755,623
4,elija ogoti obaga,['M'],1020921,934
5,antony charo mrima,['M'],1228833,506
6,bwonwong'a justus momanyi,['M'],496629,433
7,olga akech sewe,['F'],1039166,407
8,hellen wasilwa seruya,['F'],1899674,1138
9,nelly awori matheka,['F'],639133,461


In [73]:
df_thresh[50:100]

,judge,gender,tokens,cases
50,francis gikonyo,['M'],2554526,1017
51,philip nyamu waki,['M'],463868,269
52,mohammed abdullahi warsame,['M'],710036,271
53,marete d k njagi,['M'],1055456,568
54,paul kihara kariuki,['M'],232820,186
55,nicholas randa owano ombija,['M'],443336,342
56,oscar angote,['M'],1934941,1461
57,anne apondi ong'injo,['F'],323177,292
58,said juma chitembwe,['M'],1602285,1078
59,david a onyancha,['M'],1169881,830


In [74]:
df_thresh[100:150]

,judge,gender,tokens,cases
100,hedwig imbosa ong'udi,['F'],1322219,897
101,onesmus ndambuthi makau,['M'],1480993,964
102,m munyao sila,['M'],1951375,1048
103,anne omollo,['F'],959444,741
104,o a angote,['M'],86543,36
105,stephen kibunja,['M'],901375,692
106,g n wakahiu,['M'],146249,5
107,florence nyaguthii muchemi,['F'],1270137,952
108,jacqueline kamau,['F'],1862321,718
109,onguto joseph louis omondi,['M'],1002581,298


In [75]:
df_thresh[150:200]

,judge,gender,tokens,cases
150,farah s m amin,['F'],440377,171
151,john wycliffe mwera,['M'],782664,565
152,george benedict maina kariuki,['M'],516284,552
153,lucy ngima mbugua,['F'],492331,507
154,j m mutungi,['M'],54826,37
155,linnet ndolo,['F'],1091457,903
156,emmanuel okello o'kubasu,['M'],235527,209
157,w ouko,['M'],54948,55
158,john muting'a mativo,['M'],2186891,513
159,peter muchoki njoroge,['M'],1306398,1389


In [76]:
df_thresh[200:]

,judge,gender,tokens,cases
200,thripsisa wanjiku cherere,['F'],574626,506
201,festus azangalala,['M'],1265838,944
202,nagillah chrispin beda,['F'],309710,293
203,dalmas omondi ohungo,['M'],403453,279
204,william kipsiro tuiyot,['M'],88820,60
205,pamela mwikali tutui,['F'],98538,138
206,enock chacha mwita,['M'],981431,279
207,jesse nyagah njagi,['UNK'],187721,82
208,abdulhalim h athman,['M'],62141,52
209,roselyne ekirapa aburili,['F'],3463528,764


In [83]:
len(df_temp)

215